In [5]:
library(ggplot2)
library(dplyr)
library(SingleCellExperiment)

In [14]:
## input files folder
mydir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/input_files/"

In [2]:
### Neuroseq 10x data
### DA - all conditions (D30, D52, D52ROT)
### pseudo-cells (res=4)
sce = readRDS("/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/DA_phenotype_wo_outliers2_sce.rds")
sce

class: SingleCellExperiment 
dim: 32738 9219 
metadata(0):
assays(2): counts logcounts
rownames(32738): MIR1302.10 FAM138A ... AC002321.2 AC002321.1
rowData names(0):
colnames(9219): HPSI0714i-iudw_1--DA--d30--0
  HPSI0714i-iudw_1--DA--d30--1 ... HPSI0114i-rozh_4--DA--d52_tr--11
  HPSI0114i-rozh_4--DA--d52_tr--12
colData names(5): pseudocell celltype donor cluster condition
reducedDimNames(0):
spikeNames(0):
altExpNames(0):

In [9]:
## make phenotype file

In [6]:
# Phenotype (genes X pseudo-cells)
phenotype = logcounts(sce)
phenotype[1:5,1:5]

,HPSI0714i-iudw_1--DA--d30--0,HPSI0714i-iudw_1--DA--d30--1,HPSI0714i-iudw_1--DA--d30--2,HPSI0714i-iudw_1--DA--d30--3,HPSI0714i-iudw_1--DA--d30--4
MIR1302.10,0,0,0,0,0
FAM138A,0,0,0,0,0
OR4F5,0,0,0,0,0
RP11.34P13.7,0,0,0,0,0
RP11.34P13.8,0,0,0,0,0


In [32]:
rownames(phenotype)[grep('KRTAP5',rownames(phenotype))]

[1] "KRTAP5.AS1" "KRTAP5.1"   "KRTAP5.2"   "KRTAP5.3"   "KRTAP5.4"  
 [6] "KRTAP5.5"   "KRTAP5.6"   "KRTAP5.7"   "KRTAP5.8"   "KRTAP5.9"  
[11] "KRTAP5.10"  "KRTAP5.11"

In [8]:
pheno_filename = paste0(mydir,"phenotype.csv")
write.csv(phenotype, pheno_filename, quote = F)

In [ ]:
## make sample mapping file

In [10]:
head(gsub("--.*","",colnames(phenotype)))

[1] "HPSI0714i-iudw_1" "HPSI0714i-iudw_1" "HPSI0714i-iudw_1" "HPSI0714i-iudw_1"
[5] "HPSI0714i-iudw_1" "HPSI0714i-iudw_1"

In [12]:
df = data.frame(genotype_individual_id = gsub("--.*","",colnames(phenotype)),
                phenotype_sample_id = colnames(phenotype))
nrow(df)
head(df,2)

[1] 9219

genotype_individual_id,phenotype_sample_id
HPSI0714i-iudw_1,HPSI0714i-iudw_1--DA--d30--0
HPSI0714i-iudw_1,HPSI0714i-iudw_1--DA--d30--1


In [13]:
smf_filename = paste0(mydir,"sample_mapping_file.csv")
write.csv(df, smf_filename, quote = F, row.names = F)

In [ ]:
####### environments (MOFA)

In [24]:
env_filename = paste0(mydir,"10PCs.csv")
write.csv(df_pca[,1:10], env_filename, quote = F)

In [27]:
## make feature-variant filter file

In [34]:
fvf_filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/all_leads_5pct_FDR_Storey.csv"
df_fvf = read.csv(fvf_filename, row.names = 1)
head(df_fvf,2)

,snp_id,p_value,beta,beta_se,empirical_feature_p_value,feature_chromosome,feature_start,feature_end,n_samples,n_e_samples,...,snp_chromosome,snp_position,assessed_allele,call_rate,maf,hwe_p,feature_id,BH_fdr,q_value,label
2,11_107726716_A_C,4.544994e-05,0.09036565,0.02216001,0.0001665921,11,107661717,107799019,173,173,...,11,107726716,C,1,0.4306358,0.5386349,ENSG00000110660,0.001808781,0.00075681,FPP_D11
7,11_106001229_C_A,5.034677e-05,-0.24701335,0.06093060,0.0258019418,11,105946228,105969437,173,173,...,11,106001229,A,1,0.1213873,0.4744572,ENSG00000149313,0.116865230,0.04889743,FPP_D11


In [16]:
df_fvf_DA = df_fvf[grep("DA", df_fvf$label),]
nrow(df_fvf_DA)
head(df_fvf_DA,2)

[1] 2354

,snp_id,p_value,beta,beta_se,empirical_feature_p_value,feature_chromosome,feature_start,feature_end,n_samples,n_e_samples,...,snp_chromosome,snp_position,assessed_allele,call_rate,maf,hwe_p,feature_id,BH_fdr,q_value,label
9,9_36382790_A_G,3.841554e-09,0.2678636,0.04547094,1.059426e-04,9,36336393,36487545,165,165,...,9,36382790,G,1,0.2575758,0.4164226,ENSG00000137075,2.826764e-03,1.610653e-03,DA_D30
143,9_37422656_C_T,1.080706e-18,0.4319122,0.04893397,1.254483e-09,9,37422663,37436987,165,165,...,9,37422656,T,1,0.3606061,0.6138967,ENSG00000137106,1.482853e-07,8.449102e-08,DA_D30


In [17]:
unique(as.character(df_fvf_DA$label))

[1] "DA_D30"     "DA_D52"     "DA_D52_ROT"

In [18]:
DA_fvf = df_fvf_DA[,c("snp_id","feature_id")]
colnames(DA_fvf)[2] = "ensembl_gene_id"
head(DA_fvf,2)

,snp_id,ensembl_gene_id
9,9_36382790_A_G,ENSG00000137075
143,9_37422656_C_T,ENSG00000137106


In [19]:
#load map HGNC to ensembl
mapping_df = read.csv('/nfs/leia/research/stegle/dseaton/genomes/hg19/annotation/geneid_mappings/hgnc_symbol2ensembl_gene_id.txt', sep='\t')
head(mapping_df,2)

hgnc_symbol,ensembl_gene_id
SLC25A26,ENSG00000261657
,ENSG00000223116


In [20]:
nrow(DA_fvf)
DA_fvf2 = inner_join(DA_fvf, mapping_df)
nrow(DA_fvf2)

[1] 2354

Joining, by = "ensembl_gene_id"
Warning message:
“Column `ensembl_gene_id` joining factors with different levels, coercing to character vector”

[1] 2354

In [40]:
DA_fvf2$hgnc_symbol[grep('KRTAP5',DA_fvf2$hgnc_symbol)]

[1] KRTAP5-AS1 KRTAP5-AS1 KRTAP5-AS1
33671 Levels:  A1BG A1BG-AS1 A1CF A2M A2M-AS1 A2ML1 A2ML1-AS1 A2ML1-AS2 ... ZZZ3

In [25]:
head(DA_fvf2,2)

snp_id,ensembl_gene_id,hgnc_symbol,feature
9_36382790_A_G,ENSG00000137075,RNF38,RNF38
9_37422656_C_T,ENSG00000137106,GRHPR,GRHPR


In [27]:
DA_fvf2$feature = gsub("-","\\.",DA_fvf2$hgnc_symbol)

In [28]:
fvf_filename = paste0(mydir,"DA_eqtl_allconditions_FDR5pct.csv")
write.csv(DA_fvf2, fvf_filename, quote = F, row.names = F)

In [30]:
length(unique(DA_fvf2$feature))

[1] 1374